# Introduction #

The audience is a hospitality company that manages many restaurants around the US. This company wants to open a new restaurant and maximize the likelihood of the success of this restaurant. This analysis will recommend a location to open up a restaurant. This project takes on a broad scope in that is considers all major cities in the country. I will first identify the vibrant, up and coming cities as the first filter for narrowing the candidates. Of those cities, I will identify which cities may be underserved, from a gastronomic perspective. Next, I will choose one city, and identify which types of restaurants might add an exciting new element to their restaurant scene. Finally, I will conduct location-based clustering and recommend a specific cluster associated with a location to open in the target city.

# Data #

Three different data sources will be utilized. First, data from the largest cities in the US is scraped from Wikipedia. Next, the zip codes from the filtered cities is downloaded from a Zip Code API web service. Once all the zip codes for each city have been obtained, they will be appended with their corresponding latitude and longitude. Finally, the venues associated with these zip codes will be extracted from Foursquare. The data from Foursquare will be analyzed to make the final recommendation.

In [261]:
import pandas as pd
import pdb
import numpy as np
import requests
import urllib
import re
import ipdb
import collections
from geopy.geocoders import Nominatim
import geocoder
import folium # map rendering library
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # library to handle JSON files
import zipcodes
from sklearn.preprocessing import StandardScaler 
from sklearn.cluster import DBSCAN 
from sklearn.cluster import KMeans

from IPython.display import display, clear_output
from bs4 import BeautifulSoup, SoupStrainer
from googlesearch import search
pd.set_option('display.max_colwidth', 60)
import random

import zipfile
import os
from os import listdir
from os.path import isfile, join
import subprocess
from collections import Counter

The raw data from Wikipedia is as follows:

In [22]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
session = requests.Session()
response = session.get(url, allow_redirects=True)
soup = BeautifulSoup(response.content, 'html.parser')

dfx = pd.read_html(url)[4]
dfx.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York[d],New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston[3],Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


The zip code data is extracted from the API as follows:

In [23]:
zip_api_key = 'R9D3J1IzH1U8bpB0qIUx2bokJzImwCGcERHGj0QGHAZqwb0ZWbWKhh7t2juEOV9E'
call = 'https://www.zipcodeapi.com/rest/{}/city-zips.{}/{}/{}'.format(zip_api_key,'json','Chicago','Illinois')
zips = requests.get(call).json()
zips['zip_codes']

['60290',
 '60601',
 '60602',
 '60603',
 '60604',
 '60605',
 '60606',
 '60607',
 '60608',
 '60609',
 '60610',
 '60611',
 '60612',
 '60613',
 '60614',
 '60615',
 '60616',
 '60617',
 '60618',
 '60619',
 '60620',
 '60621',
 '60622',
 '60623',
 '60624',
 '60625',
 '60626',
 '60628',
 '60629',
 '60630',
 '60631',
 '60632',
 '60633',
 '60634',
 '60636',
 '60637',
 '60638',
 '60639',
 '60640',
 '60641',
 '60642',
 '60643',
 '60644',
 '60645',
 '60646',
 '60647',
 '60649',
 '60651',
 '60652',
 '60653',
 '60654',
 '60655',
 '60656',
 '60657',
 '60659',
 '60660',
 '60661',
 '60663',
 '60664',
 '60666',
 '60668',
 '60669',
 '60670',
 '60673',
 '60674',
 '60675',
 '60677',
 '60678',
 '60679',
 '60680',
 '60681',
 '60682',
 '60684',
 '60685',
 '60686',
 '60687',
 '60688',
 '60689',
 '60690',
 '60691',
 '60693',
 '60694',
 '60695',
 '60696',
 '60697',
 '60699',
 '60701',
 '60706',
 '60707',
 '60712',
 '60803',
 '60804',
 '60805',
 '60827']

# Methodology #

Data is first scraped from Wikipedia.

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
session = requests.Session()
response = session.get(url, allow_redirects=True)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
test = pd.read_html(url)[4]

In [5]:
test.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York[d],New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston[3],Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


Data cleaning is conducted below. The three columns below parsed and converted to numeric columns and State column is renamed.

In [6]:
test["Change"] = list(map(lambda x: x[1:-1], test["Change"]))
test["Change"] = pd.to_numeric(test["Change"], errors='coerce')
test["2010Census"] = pd.to_numeric(test["2010Census"], errors='coerce')
test["2018estimate"] = pd.to_numeric(test["2018estimate"], errors='coerce')
test.rename(columns = {"State[c]":"State"},inplace=True)

Next, the population density and land area are parsed and converted to numeric columns.

In [7]:
test['2016 Population Density (people/km2)'] = list(map(lambda x: float(x.split("/km")[0].replace(",","")), test['2016 population density.1']))
test['2016 Land Area (km2)'] = list(map(lambda x: float(x.split()[0].replace(",","")), test['2016 land area.1']))

The latitude and longitude columns are parsed so they can be used to extract venues.

In [8]:
lat_split = list(map(lambda x: x.split("/")[1].split()[0],test["Location"]))
long_split = list(map(lambda x: x.split("/")[1].split()[1],test["Location"]))
test["Latitude"] = list(map(lambda x: float(re.search("[0-9.]+",x).group(0)),lat_split))
test["Longitude"] = list(map(lambda x: float(re.search("[0-9.]+",x).group(0)),long_split))

City names are also cleaned up.

In [9]:
test["City"] = list(map(lambda x: x.split("[")[0],test["City"]))

In [138]:
test.head(10)

,2018rank,City,State,2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location,2016 Population Density (people/km2),2016 Land Area (km2),Latitude,Longitude
0,1,New York,New York,8398748,8175133,2.74,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W,10933.0,780.9,40.6635,73.9387
1,2,Los Angeles,California,3990456,3792621,5.22,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W,3276.0,1213.9,34.0194,118.4108
2,3,Chicago,Illinois,2705994,2695598,0.39,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W,4600.0,588.7,41.8376,87.6818
3,4,Houston,Texas,2325502,2100263,10.72,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W,1395.0,1651.1,29.7866,95.3909
4,5,Phoenix,Arizona,1660272,1445632,14.85,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W,1200.0,1340.6,33.5722,112.0901
5,6,Philadelphia,Pennsylvania,1584138,1526006,3.81,134.2 sq mi,347.6 km2,"11,683/sq mi","4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W,4511.0,347.6,40.0094,75.1333
6,7,San Antonio,Texas,1532233,1327407,15.43,461.0 sq mi,"1,194.0 km2","3,238/sq mi","1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W,1250.0,1194.0,29.4724,98.5251
7,8,San Diego,California,1425976,1307402,9.07,325.2 sq mi,842.3 km2,"4,325/sq mi","1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°W,1670.0,842.3,32.8153,117.1350
8,9,Dallas,Texas,1345047,1197816,12.29,340.9 sq mi,882.9 km2,"3,866/sq mi","1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W,1493.0,882.9,32.7933,96.7665
9,10,San Jose,California,1030119,945942,8.90,177.5 sq mi,459.7 km2,"5,777/sq mi","2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°W,2231.0,459.7,37.2967,121.8189


Unused columns are removed.

In [24]:
city_df = test.drop(test.columns[6:11],axis=1)
city_df.head()

,2018rank,City,State,2018estimate,2010Census,Change,2016 Population Density (people/km2),2016 Land Area (km2),Latitude,Longitude
0,1,New York,New York,8398748,8175133,2.74,10933.0,780.9,40.6635,73.9387
1,2,Los Angeles,California,3990456,3792621,5.22,3276.0,1213.9,34.0194,118.4108
2,3,Chicago,Illinois,2705994,2695598,0.39,4600.0,588.7,41.8376,87.6818
3,4,Houston,Texas,2325502,2100263,10.72,1395.0,1651.1,29.7866,95.3909
4,5,Phoenix,Arizona,1660272,1445632,14.85,1200.0,1340.6,33.5722,112.0901


The existing latitude and longitude columns are incorrect, so geolocator is used to update these columns.

In [12]:
lats, longs = [],[]

for city, state in zip(test["City"],test["State"]):
    address = city + ", " + state

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lats.append(latitude)
    longs.append(longitude)
    print('The geograpical coordinate of {} are {}, {}.'.format(city,latitude, longitude))

The geograpical coordinate of New York are 40.7127281, -74.0060152.
The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.
The geograpical coordinate of Chicago are 41.8755616, -87.6244212.
The geograpical coordinate of Houston are 29.7589382, -95.3676974.
The geograpical coordinate of Phoenix are 33.4484367, -112.0741417.
The geograpical coordinate of Philadelphia are 39.9527237, -75.1635262.
The geograpical coordinate of San Antonio are 29.4246002, -98.4951405.
The geograpical coordinate of San Diego are 32.7174209, -117.1627714.
The geograpical coordinate of Dallas are 32.7762719, -96.7968559.
The geograpical coordinate of San Jose are 37.3361905, -121.8905833.
The geograpical coordinate of Austin are 30.2711286, -97.7436995.
The geograpical coordinate of Jacksonville are 30.3321838, -81.655651.
The geograpical coordinate of Fort Worth are 32.753177, -97.3327459.
The geograpical coordinate of Columbus are 39.9622601, -83.0007065.
The geograpical coordinate of San Fr

In [146]:
city_df["Latitude"] = lats
city_df["Longitude"] = longs

In [147]:
city_df.head(10)

,2018rank,City,State,2018estimate,2010Census,Change,2016 Population Density (people/km2),2016 Land Area (km2),Latitude,Longitude
0,1,New York,New York,8398748,8175133,2.74,10933.0,780.9,40.712728,-74.006015
1,2,Los Angeles,California,3990456,3792621,5.22,3276.0,1213.9,34.053691,-118.242767
2,3,Chicago,Illinois,2705994,2695598,0.39,4600.0,588.7,41.875562,-87.624421
3,4,Houston,Texas,2325502,2100263,10.72,1395.0,1651.1,29.758938,-95.367697
4,5,Phoenix,Arizona,1660272,1445632,14.85,1200.0,1340.6,33.448437,-112.074142
5,6,Philadelphia,Pennsylvania,1584138,1526006,3.81,4511.0,347.6,39.952724,-75.163526
6,7,San Antonio,Texas,1532233,1327407,15.43,1250.0,1194.0,29.424600,-98.495141
7,8,San Diego,California,1425976,1307402,9.07,1670.0,842.3,32.717421,-117.162771
8,9,Dallas,Texas,1345047,1197816,12.29,1493.0,882.9,32.776272,-96.796856
9,10,San Jose,California,1030119,945942,8.90,2231.0,459.7,37.336191,-121.890583


In [148]:
dallas = city_df[city_df["City"] == "Dallas"]

In [149]:
dallas

,2018rank,City,State,2018estimate,2010Census,Change,2016 Population Density (people/km2),2016 Land Area (km2),Latitude,Longitude
8,9,Dallas,Texas,1345047,1197816,12.29,1493.0,882.9,32.776272,-96.796856


The statistics of the data frame are used to determine the filter thresholds.

In [150]:
city_df.describe()

,2018rank,2018estimate,2010Census,Change,2016 Population Density (people/km2),2016 Land Area (km2),Latitude,Longitude
count,314.000000,3.140000e+02,3.140000e+02,313.000000,314.000000,314.000000,314.000000,314.000000
mean,157.500000,3.023532e+05,2.796192e+05,9.467700,1617.716561,242.099682,36.433751,-98.859597
std,90.788215,5.865428e+05,5.628653e+05,8.885156,1278.077415,351.681812,5.238434,17.148916
min,1.000000,1.001540e+05,0.000000e+00,0.020000,68.000000,16.600000,21.304547,-157.855676
25%,79.250000,1.205110e+05,1.100615e+05,3.280000,888.250000,87.500000,33.196315,-117.377193
50%,157.500000,1.661260e+05,1.525620e+05,7.880000,1330.500000,143.000000,36.041356,-96.704051
75%,235.750000,2.642332e+05,2.377558e+05,12.160000,1840.500000,270.475000,40.437099,-83.542647
max,314.000000,8.398748e+06,8.175133e+06,67.590000,10933.000000,4420.100000,61.216313,-71.058291


One of the primary aspects of this rationale is that the city must already be large to ensure there is a large potential consumer base. The 75th percentile of 230000 people is determined to be too small, therefore I set the threshold of 50000 people. Additionally, since these are growing cities the density threshold is set lower, at the 25th threshold. The data set is first filtered by this threshold. Finally, the most important consideration is that this city is growing quickly, therefore, the filtered dataframe is ordered by growth rate and the top 5 cities are examined.

In [369]:
city_filter = city_df[(city_df["2018estimate"] > 500000) & (city_df["2016 Population Density (people/km2)"] > 888.25)]
target_city_df = city_filter.sort_values(by = "Change", ascending=False).head(5)
target_city_df

,2018rank,City,State,2018estimate,2010Census,Change,2016 Population Density (people/km2),2016 Land Area (km2),Latitude,Longitude
17,18,Seattle,Washington,744955,608660,22.39,3245.0,217.0,47.603832,-122.330062
10,11,Austin,Texas,964254,790390,22.00,1170.0,809.9,30.271129,-97.743700
12,13,Fort Worth,Texas,895008,741206,20.75,962.0,888.1,32.753177,-97.332746
18,19,Denver,Colorado,716492,600158,19.38,1746.0,397.0,39.739236,-104.984862
15,16,Charlotte,North Carolina,872498,731424,19.29,1064.0,791.0,35.227087,-80.843127


In [155]:
target_city_df = target_city_df.append(dallas)

The five cities analyzed are Seattle, Austin, Fort Worth, Denver and Charlotte. Two state names are converted to their abbreviations so the api can return the zip codes.

In [156]:
target_city_df["State"].replace("Texas","TX",inplace=True)
target_city_df["State"].replace("North Carolina","NC",inplace=True)

In [157]:
df_rows = []
zip_api_key = 'R9D3J1IzH1U8bpB0qIUx2bokJzImwCGcERHGj0QGHAZqwb0ZWbWKhh7t2juEOV9E'

for index, row in target_city_df[["City","State"]].iterrows():
    print("Extracting Data for {}, {}" .format(row["City"], row["State"]))
    call = 'https://www.zipcodeapi.com/rest/{}/city-zips.{}/{}/{}'.format(zip_api_key,'json',row["City"],row["State"])
    zip_response = requests.get(call).json()
    
    for city_zip in zip_response['zip_codes']:
        df_rows.append({"City":row["City"], "State":row["State"], "Zip_Code":city_zip})
        
city_data = pd.DataFrame(df_rows)

Extracting Data for Seattle, Washington
Extracting Data for Austin, TX
Extracting Data for Fort Worth, TX
Extracting Data for Denver, Colorado
Extracting Data for Charlotte, NC
Extracting Data for Dallas, TX


The number of zip codes per city are as follows:

In [158]:
city_data.groupby("City").count()

,State,Zip_Code
City,,
Austin,81,81
Charlotte,74,74
Dallas,121,121
Denver,80,80
Fort Worth,60,60
Seattle,65,65


The package zipcodes was used to update each zip code with a latitude and longitude.

In [159]:
target_latitudes, target_longitudes = [], [] 

for index,row in city_data.iterrows():  
    target_latitudes.append(zipcodes.matching(row["Zip_Code"])[0]['lat'])
    target_longitudes.append(zipcodes.matching(row["Zip_Code"])[0]['long'])
    
city_data["Latitude"] = target_latitudes
city_data["Longitude"] = target_longitudes

Some duplicates were found and these were removed.

In [161]:
print("There are {} unique Lat Long Coordinates for Seattle".format(len(city_data[city_data["City"] == "Seattle"]["Latitude"].unique())))
print("There are {} unique Lat Long Coordinates for Austin".format(len(city_data[city_data["City"] == "Austin"]["Latitude"].unique())))
print("There are {} unique Lat Long Coordinates for Charlotte".format(len(city_data[city_data["City"] == "Charlotte"]["Latitude"].unique())))
print("There are {} unique Lat Long Coordinates for Fort Worth".format(len(city_data[city_data["City"] == "Fort Worth"]["Latitude"].unique())))
print("There are {} unique Lat Long Coordinates for Denver".format(len(city_data[city_data["City"] == "Denver"]["Latitude"].unique())))

There are 46 unique Lat Long Coordinates for Seattle
There are 51 unique Lat Long Coordinates for Austin
There are 34 unique Lat Long Coordinates for Charlotte
There are 40 unique Lat Long Coordinates for Fort Worth
There are 59 unique Lat Long Coordinates for Denver


In [162]:
target_df = city_data.drop_duplicates(subset="Latitude")
target_df.head()

,City,State,Zip_Code,Latitude,Longitude
0,Seattle,Washington,98101,47.6110,-122.3335
1,Seattle,Washington,98102,47.6357,-122.3244
2,Seattle,Washington,98103,47.6703,-122.3483
3,Seattle,Washington,98104,47.6021,-122.3284
4,Seattle,Washington,98105,47.6604,-122.2805


The total number of geographical locations for each city is as follows.

In [163]:
target_df.groupby("City").count()

,State,Zip_Code,Latitude,Longitude
City,,,,
Austin,51,51,51,51
Charlotte,34,34,34,34
Dallas,63,63,63,63
Denver,59,59,59,59
Fort Worth,40,40,40,40
Seattle,46,46,46,46


These next two functions query Foursquare to retrieve all the venues near the coordinates provided for each zip code. This data is cleaned and uploaded to a dataframe.

In [164]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [166]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip_Code', 
                  'Zip_Code Latitude', 
                  'Zip_Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Foursquare Credentials are loaded.

In [200]:
CLIENT_ID = 'VO5EZOO3GJMZV02BM0PZ4J4A4DRM3OMF0KTF2UOP0BPN1YAB' # your Foursquare ID
CLIENT_SECRET = '1CXWD2105GLMT5BL4CLAZXMOCUN2FBSYGTY2GPV0YMZ4NXY5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 5000
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VO5EZOO3GJMZV02BM0PZ4J4A4DRM3OMF0KTF2UOP0BPN1YAB
CLIENT_SECRET:1CXWD2105GLMT5BL4CLAZXMOCUN2FBSYGTY2GPV0YMZ4NXY5


In [201]:
target_venues = getNearbyVenues(names=target_df['Zip_Code'], latitudes=target_df['Latitude'], longitudes=target_df['Longitude'])

98101
98102
98103
98104
98105
98106
98107
98108
98109
98111
98112
98113
98115
98116
98117
98118
98119
98121
98125
98126
98131
98132
98133
98134
98136
98144
98146
98148
98151
98154
98155
98158
98160
98161
98164
98166
98168
98174
98177
98178
98188
98189
98195
98198
98199
98110
73301
73344
78701
78702
78703
78704
78705
78710
78712
78717
78719
78721
78722
78723
78724
78725
78726
78727
78728
78729
78730
78731
78732
78733
78734
78735
78736
78737
78738
78739
78741
78742
78744
78745
78746
78747
78748
78749
78750
78751
78752
78753
78754
78755
78756
78757
78758
78759
78769
78786
78798
76101
76102
76103
76104
76105
76106
76107
76108
76109
76110
76111
76112
76114
76115
76116
76118
76119
76120
76122
76123
76126
76129
76131
76132
76133
76134
76135
76137
76140
76148
76155
76164
76166
76177
76179
76117
76127
76180
76182
76244
80201
80202
80203
80204
80205
80206
80207
80208
80209
80210
80211
80212
80214
80215
80216
80218
80219
80220
80221
80222
80223
80224
80225
80226
80227
80228
80229
80230
80231
8023

In [202]:
total_data_df = target_venues.merge(target_df,on="Zip_Code")
total_df = total_data_df.drop(["State","Latitude","Longitude"],axis=1)
total_df.head()

,Zip_Code,Zip_Code Latitude,Zip_Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City
0,98101,47.6110,-122.3335,ACT Theatre,47.610763,-122.332905,Theater,Seattle
1,98101,47.6110,-122.3335,Monorail Espresso,47.610828,-122.335048,Coffee Shop,Seattle
2,98101,47.6110,-122.3335,The 5th Avenue Theatre,47.608996,-122.334162,Theater,Seattle
3,98101,47.6110,-122.3335,MOD Pizza,47.609490,-122.332647,Pizza Place,Seattle
4,98101,47.6110,-122.3335,The Cheesecake Factory,47.611625,-122.333234,American Restaurant,Seattle


Here is the number of venues by city:

In [203]:
pd.DataFrame(total_df.groupby("City").count()["Venue Category"])

,Venue Category
City,
Austin,695
Charlotte,728
Dallas,1039
Denver,1017
Fort Worth,203
Seattle,994


Visual inspection of the unique venues resulted in this list used to categories venues as a food venue.

In [204]:
food_categories = ['coffee shop','place','restaurant','bar','steakhouse','tea','breakfast','lunch','dinner','café','joint','donut',
                   'noodle house','food','drink','pub','speakeasy','gastropub','diner','market']

The food category indicator is assigned the dataframe of venues.

In [370]:
indicator = []

for index,row in total_df.iterrows():
    
    if any(place in row["Venue Category"].lower() for place in food_categories):
        indicator.append(1)
    else:
        indicator.append(0)
        
total_df['Food_Category'] = indicator
total_df.head()

,Zip_Code,Zip_Code Latitude,Zip_Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City,Food_Category
0,98101,47.6110,-122.3335,ACT Theatre,47.610763,-122.332905,Theater,Seattle,0
1,98101,47.6110,-122.3335,Monorail Espresso,47.610828,-122.335048,Coffee Shop,Seattle,1
2,98101,47.6110,-122.3335,The 5th Avenue Theatre,47.608996,-122.334162,Theater,Seattle,0
3,98101,47.6110,-122.3335,MOD Pizza,47.609490,-122.332647,Pizza Place,Seattle,1
4,98101,47.6110,-122.3335,The Cheesecake Factory,47.611625,-122.333234,American Restaurant,Seattle,1


Fort Worth is clearly the most underserved area by a large margin of about 6%. However, Fort Worth is very close to Dallas. If Dallas had a glut of restaurants, Fort Worth could lose potential customers to Dallas restaurants. Therefore, the restaurant composition of Dallas is also analyzed. Dallas appears to fall roughly in the middle for percentage of restaurants, therefore, the proximity of Dallas will not deter me from recommending Fort Worth.

In [208]:
total_df.groupby("City").mean()

,Venue Latitude,Venue Longitude,Food_Category
City,,,
Austin,30.292837,-97.740343,0.528058
Charlotte,35.221678,-80.837212,0.579670
Dallas,32.801533,-96.815146,0.498556
Denver,39.744781,-104.988373,0.474926
Fort Worth,32.766511,-97.327082,0.418719
Seattle,47.614814,-122.335455,0.483903


The Fort Worth data is extracted from the whole data frame.

In [242]:
fw_df = total_df[total_df["City"] == "Fort Worth"]
fort_worth = fw_df.reset_index(drop=True)

In [243]:
fort_worth.head()

,Zip_Code,Zip_Code Latitude,Zip_Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,City,Food_Category
0,76101,32.7254,-97.3208,Shell,32.725692,-97.319880,Gas Station,Fort Worth,0
1,76101,32.7254,-97.3208,Allen Food Store,32.726245,-97.318192,Convenience Store,Fort Worth,0
2,76101,32.7254,-97.3208,Arlington Heights Pet Sitters,32.725758,-97.317560,Pet Store,Fort Worth,0
3,76101,32.7254,-97.3208,food,32.727272,-97.318069,BBQ Joint,Fort Worth,1
4,76101,32.7254,-97.3208,KELDEN COMPANIES,32.723700,-97.317291,Construction & Landscaping,Fort Worth,0


In [244]:
latitude = target_city_df[target_city_df["City"] == "Fort Worth"]["Latitude"]
longitude = target_city_df[target_city_df["City"] == "Fort Worth"]["Longitude"]

map_fort_worth = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, zip_code, venue in zip(fort_worth['Venue Latitude'], fort_worth['Venue Longitude'], fort_worth['Zip_Code'],fort_worth['Venue Category']):
    label = '{}, {}'.format(venue, zip_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_fort_worth)  
    
map_fort_worth

The Fort Worth venues are plotted above. Note, these venues seem sparsely located. Consulting additional data would allieve this uncertainty.

Next, location data is fed into the KMeans Clustering algorithm to separate the venues into 8 clusters. The value of 8 was chosen such that the central part of the city would be divided into a few clusters to provide multiple options for the location of this restaurant.

In [280]:
input_data = np.array(fort_worth[["Venue Latitude","Venue Longitude"]])
# set number of clusters
kclusters = 8

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(input_data)

fort_worth['Cluster Labels'] = kmeans.labels_

In [281]:
latitude = target_city_df[target_city_df["City"] == "Fort Worth"]["Latitude"]
longitude = target_city_df[target_city_df["City"] == "Fort Worth"]["Longitude"]

kclusters = len(set(labels))

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(fort_worth['Venue Latitude'], fort_worth['Venue Longitude'], fort_worth['Zip_Code'], fort_worth['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

I want the restaurant to be relatively close to downtown in order to be reachable by the maximum number of consumers, therefore, I will consider clusters 1,2,3,5, and 7

In [282]:
fort_worth.groupby("Cluster Labels").mean()

,Venue Latitude,Venue Longitude,Food_Category
Cluster Labels,,,
0,32.925880,-97.247975,0.470588
1,32.705888,-97.365643,0.428571
2,32.750462,-97.326982,0.465517
3,32.761821,-97.190608,0.384615
4,32.640307,-97.544317,0.000000
5,32.758766,-97.446711,0.238095
6,32.617822,-97.364491,0.466667
7,32.818925,-97.275608,0.411765


Based on the breakdown of food venues by cluster, cluster 5 appears to be the most underserved. Next, I will decide what type of restaurant to recommend. I will determine the least common restaurants by cluster and for all of Fort Worth to identify any overlap. For the sake of simplicity, I will not consider recommending a restaurant that is not found in Fort Worth.

In [285]:
# one hot encoding
fort_worth_onehot = pd.get_dummies(fort_worth[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
fort_worth_onehot['Cluster Labels'] = fort_worth['Cluster Labels'] 

# Find the neighborhood columns and move it to the front
cols = fort_worth_onehot.columns.tolist()
first_column = ['Cluster Labels']
new_order = first_column + cols[:cols.index('Cluster Labels')] + cols[cols.index('Cluster Labels') + 1 :]
fort_worth_onehot = fort_worth_onehot[new_order]

fort_worth_onehot.head()

,Cluster Labels,American Restaurant,Art Gallery,Arts & Crafts Store,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,...,Steakhouse,Supermarket,Supplement Shop,Taco Place,Theater,Thrift / Vintage Store,Trail,Turkish Restaurant,Video Game Store,Wine Bar
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [288]:
fw_grouped = fort_worth_onehot.groupby('Cluster Labels').mean().reset_index()
fw_grouped.head()

,Cluster Labels,American Restaurant,Art Gallery,Arts & Crafts Store,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,...,Steakhouse,Supermarket,Supplement Shop,Taco Place,Theater,Thrift / Vintage Store,Trail,Turkish Restaurant,Video Game Store,Wine Bar
0,0,0.029412,0.058824,0.029412,0.00000,0.029412,0.058824,0.00000,0.029412,0.000000,...,0.029412,0.029412,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.00000
1,1,0.000000,0.000000,0.000000,0.02381,0.000000,0.000000,0.02381,0.023810,0.071429,...,0.000000,0.000000,0.0,0.02381,0.000000,0.0,0.023810,0.000000,0.0,0.02381
2,2,0.051724,0.000000,0.000000,0.00000,0.000000,0.051724,0.00000,0.017241,0.000000,...,0.000000,0.000000,0.0,0.00000,0.017241,0.0,0.017241,0.017241,0.0,0.00000
3,3,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.00000
4,4,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.00000


This computes the frequency of different venues for all of Fort Worth.

In [360]:
total_venues = pd.DataFrame(fort_worth_onehot.sum(axis=0)).T
total_venues_avg = total_venues / total_venues[total_venues.columns[1:]].sum(axis=1)[0]
total_venues_avg

,Cluster Labels,American Restaurant,Art Gallery,Arts & Crafts Store,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,...,Steakhouse,Supermarket,Supplement Shop,Taco Place,Theater,Thrift / Vintage Store,Trail,Turkish Restaurant,Video Game Store,Wine Bar
0,2.576355,0.024631,0.009852,0.004926,0.004926,0.004926,0.029557,0.004926,0.014778,0.019704,...,0.004926,0.004926,0.004926,0.004926,0.004926,0.009852,0.009852,0.004926,0.004926,0.004926


This function returns the least common venues with at least one occurence.

In [309]:
def return_least_common_venues(row, num_top_venues):
#   filter out venues that don't appear  
    row_categories = row.iloc[1:]
    row_categories = row_categories[row_categories > 0]
    row_categories_sorted = row_categories.sort_values(ascending=True)
#     print(row, row_categories_sorted)
    return row_categories_sorted.index.values[0:num_top_venues]

In [354]:
num_top_venues = 10
least = []

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cluster Labels']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Least Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Least Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Cluster Labels'] = fw_grouped['Cluster Labels']

for ind in np.arange(fw_grouped.shape[0]):
    least.append(return_least_common_venues(fw_grouped.iloc[ind, :], num_top_venues))

The least common venues for all 8 clusters are found below. We can see 4 different restaurants appear in this cluster.

In [358]:
pd.DataFrame(least,columns=columns[1:])

,1st Least Common Venue,2nd Least Common Venue,3rd Least Common Venue,4th Least Common Venue,5th Least Common Venue,6th Least Common Venue,7th Least Common Venue,8th Least Common Venue,9th Least Common Venue,10th Least Common Venue
0,American Restaurant,Shoe Store,Pool,Pizza Place,Pet Store,Outdoor Supply Store,Miscellaneous Shop,Laundromat,Gym / Fitness Center,Gym
1,Auto Garage,Taco Place,Smoothie Shop,Pharmacy,Ice Cream Shop,Home Service,Gym / Fitness Center,Gym,Fried Chicken Joint,Food Truck
2,Turkish Restaurant,Pizza Place,Performing Arts Venue,Nature Preserve,Museum,Movie Theater,Ice Cream Shop,Home Service,Gym / Fitness Center,Trail
3,Bookstore,Clothing Store,Discount Store,Gas Station,Grocery Store,Home Service,Insurance Office,Locksmith,Mexican Restaurant,Sandwich Place
4,Furniture / Home Store,Locksmith,None,None,None,None,None,None,None,None
5,BBQ Joint,Seafood Restaurant,Rental Car Location,Pizza Place,Pawn Shop,Park,Movie Theater,Japanese Restaurant,Gym / Fitness Center,Grocery Store
6,American Restaurant,Bank,Construction & Landscaping,Convenience Store,General Entertainment,Hobby Shop,Italian Restaurant,Mexican Restaurant,Nail Salon,Pizza Place
7,Baseball Field,Construction & Landscaping,Diner,Furniture / Home Store,Mexican Restaurant,Park,Playground,Sandwich Place,Video Game Store,Discount Store


Next, I look at the frequency of the least common venues of cluster 5 for all of Fort Worth. I see that of the 4 types of restaurants identified in cluster 5, Japanese Restaurants are the least common type of restaurant in Fort Worth as a whole. Therefore, I recommend opening a Japanese restaurant.

In [359]:
total_venues_avg.T.loc[least[5],:].sort_values(by=0,ascending=True)

,0
Rental Car Location,0.004926
Pawn Shop,0.004926
Japanese Restaurant,0.004926
Movie Theater,0.009852
Grocery Store,0.009852
Seafood Restaurant,0.019704
Gym / Fitness Center,0.019704
BBQ Joint,0.029557
Park,0.034483
Pizza Place,0.049261


In [365]:
print("The zip codes comprising cluster 5 are {}" .format(fort_worth[fort_worth["Cluster Labels"] == 5]["Zip_Code"].unique().tolist()))

The zip codes comprising cluster 5 are ['76114', '76116', '76135', '76179', '76127']


# Results and Discussion #

Based on the results of the analysis I recommend opening a Japanese Restaurant in Fort Worth. Specifically, I recommend opening this restaurant within one of 5 zip codes: 76114, 76116, 76135, 76179, 76127. This conclusion was reached as a result of identifying that Japanese restaurants had a small presence in cluster 5 and in Fort Worth as whole. It is important to note the limitations and uncertainties of this analysis. The primary source of uncertainty was the number of venues returned by Fort Worth. Fort Worth returned 3 to 5 times less venues than the other cities considered. This confidence depends on the confidence we have in the foursquare data. We are confident in the foursquare data so it is possible we missed zip codes that would have provided more venues. Additionally, the consideration of the proximity of Fort Worth to Dallas. Dallas was included in the restaurant ratio analysis because I determined if Dallas as overserved from a restaurant perspective, that would distort the low ratio for Fort Worth because restaurants in Fort Worth would have to compete with restaurants in Dallas.

In [367]:
total_df.groupby("City").mean()[["Food_Category"]]

,Food_Category
City,
Austin,0.528058
Charlotte,0.579670
Dallas,0.498556
Denver,0.474926
Fort Worth,0.418719
Seattle,0.483903


Given Dallas, fell in the middle of the pack, I determined Dallas to be a wash and not affect the decision to choose Fort Worth. I decided to use a location based Clustering algorithm because my recommendation involved the physical location within Fort Worth. Therefore, I used KMeans to group the venues by location. I chose 8 clusters in order to provide multiple options for the location of this restaurant. Note, this choice was relatively arbitrary and could have benefitted from additional scrutiny. Doubts about sufficient venue data was the primary reason the number of clusters was not scrutinized further. Finally, I needed to decide what type of restaurant to recommend to build in cluster 5. I made the simplifying assumption not to recommend a type of restaurant that was not already present in Fort Worth. I also assumed the residents of Fort Worth have roughly as similar preference for all restaurants in Fort Worth. This is obviously a huge, unrealistic assumption, however, significant steps would have to be taken to ascertain the restaurant preferences of Fort Worth residents. This would require some sort of survey or restaurant performance data to determine which restaurants perform the best in Fort Worth. Given my assumptions, I looked for restaurants that were underrepresented in Cluster 5 and Fort Worth as a whole. The restaurant of type "Japanese" satified this union of low occurence in cluster 5 and Fort Worth overall. This union was decided so this restaurant would have less competition to consumers in cluster 5 and Fort Worth. It must be noted that there is reasonable possibility that a type of restaurant has a low number of occurences because this type of restaurant may be unfavorable to its residents. However, this cannot be concluded with more data.

# Conclusion #

This analysis consisting of filtering the largest cities in America and identifying the 5 fastest growing cities that contained as least 500000 residents and met the 25th percentile of population density within potential cities. The cities I identified were Seattle, Charlotte, Austin, Fort Worth and Denver. For each city, I obtained a list of Zip codes by accessing a Zip Code API. Next, I used a python package called "zipcodes" to add a latitude and longitude to each zip code. Using these coordinates, I accessed foursquare to return all the venues near these coordinates. I defined a list of venues which fell into the criteria of food. I then determined what percentage of the total venues were food venues for each city. Fort Worth contained the lowest ratio, at about 41%, therefore, I chose this city as the target. I took all the venues and used KMeans to create 8 clusters. I identified 5 clusters near the center of the city to determine was the final location for the restaurant. I again determined the food venues as a percentage of the total venues for each cluster to determine the optimal location for this restaurant within Fort Worth. I identified the fifth cluster as containing the least amount of restaurants, at 23.8%. Next, I determined the least frequently occurring venues with at least one occurence in Cluster 5 and Fort Worth as a whole. I found Japanese Restaurants as one of the least frequently occuring food category in both subsets, therefore, I made the recommendation to open a Japanese Restaurant within Cluster 5 in Fort Worth, Texas.